# Data Collector
In this notebook, we collect the simulation data on qulacs.

## QRAO on regular graphs

In [1]:
# import packages
from maxcut_instance_generator import regular_graph
from encoding import RandomAccessEncoder
from vqe import VQEForQRAO
from rounding import MagicRounding

In [27]:
import os
import pickle
from tqdm.auto import tqdm

# function to run QRAO
def run_qrao(m, n, instance, max_level, root_path, trial, shots):
    os.makedirs(root_path, exist_ok=True)

    qrac = RandomAccessEncoder(m, n)
    hamiltonian = qrac.generate_hamiltonian(instance)
    # print(f"Hamiltonian is {hamiltonian}")
    num_qubit = len(qrac.qubit_to_vertex_map)
    num_edge = len(qrac.calculate_edge_among_qubits(instance))
    # print(f"{num_qubit} qubits, {num_edge} edges")

    for entanglement in ["compatible", "linear", "random"]:
        for level in range(max_level):
            vqe = VQEForQRAO(
                hamiltonian,
                entanglement=entanglement,
                num_layer=level,
                qubit_pairs=qrac.calculate_edge_among_qubits(instance),
            )
            cost_history, best_theta_list = vqe.minimize()
            rounding = MagicRounding(m, n, shots, vqe, qrac)
            solution_counts = rounding.round(best_theta_list)
            maxcut_values = rounding.get_objective_value_counts(
                instance, solution_counts
            )

            # result of the experiment
            result = {
                # "solution_counts": solution_counts,
                "maxcut_values": maxcut_values,
                "num_qubit": num_qubit,
                "num_edge": num_edge,
                "entanglement": entanglement,
                "level": level,
                "optimum_solution": instance.solve().get_objective_value(),
            }

            # save experiment result
            save_file_name = (
                f"{root_path}/{m}_{n}/{entanglement}/level{level}_trial{trial}.pkl"
            )
            with open(save_file_name, "wb") as f:
                pickle.dump(result, f)

In [28]:
# search pattern
# search_pattern = {3: [8, 12, 16, 20, 24, 28, 32, 36, 40]}
search_pattern = {3: [12]}

MAX_LEVEL = 4
TRIAL = 1
ROUNDING_SHOTS = 10000

In [29]:
for deg, num_vertices in tqdm(search_pattern.items()):
    for num in num_vertices:
        for i in range(TRIAL):
            instance = regular_graph(num, deg)
            run_qrao(
                3,
                1,
                instance,
                MAX_LEVEL,
                f"results/regular/deg{deg}/nodes{num}",
                i,
                ROUNDING_SHOTS,
            )

Iter	cost
1/25000	-6.5060754391482805
2/25000	-7.023490427079777
3/25000	-7.06891645012167
4/25000	-4.506529625622496
5/25000	-7.133952115772491
6/25000	-7.133952115772491
7/25000	-7.731309481851555
8/25000	-6.587247867263789
9/25000	-7.731309481851553
10/25000	-7.686254108616502
11/25000	-7.733957469215845
12/25000	-7.733957469215844
13/25000	-6.413071869746383
14/25000	-7.602695470347625
15/25000	-7.733957469215843
16/25000	-6.390203762199867
17/25000	-7.8016261977729595
18/25000	-7.8016261977729595
19/25000	-10.16848806079022
20/25000	-10.74594343510501
21/25000	-10.65219105611039
22/25000	-10.734777610892767
23/25000	-10.517922121970987
24/25000	-10.745943435105014
25/25000	-11.29955673419385
26/25000	-10.816313069386005
27/25000	-11.256617201500758
28/25000	-11.29955673419385
29/25000	-10.523199268362834
30/25000	-11.133925825115632
31/25000	-11.223196420200285
32/25000	-11.29955673419385
33/25000	-10.3309103350555
34/25000	-11.961077816603618
35/25000	-13.271199880188071
36/25000